In [ ]:
from pysmt.shortcuts import *
import pysmt.typing as types
import random as rn
from pysmt.typing import BOOL, REAL, INT, BVType, STRING
from IPython.display import Latex
import itertools

# Constantes
a = 0.01  # atrito nos restantes modos
a_blocked = 0.009 # atrito no modo blocked
c_stopping = 10.0  # proporcionalidade de travagem em stopping
c = 0.5  # proporcionalidade de travegem nos restantes modos
b = 0.01 # atrito no contacto corpo/ar
P = 1000.0  # peso do veículo
tau = 0.1  # limite máximo do contador nos modos free e blocked
dt = 0.1  # diferença da variavél tempo nas transições timed
V0 = 20.0  # velocidade inicial do veículo
erro = 0.2  # para compensar possíveis erros pela falta de rigor da fórmulas do programa e evitar trajetórias de Zenão

# Modos
START = Int(0)
FREE = Int(1)
STOPPING = Int(2)
BLOCKED = Int(3)
STOPPED = Int(4)

# Função declare
def declare(i):
    s = {}
    s['T'] = Symbol('T'+str(i), REAL)
    s['V'] = Symbol('V'+str(i), REAL)
    s['v'] = Symbol('v'+str(i), REAL)
    s['M'] = Symbol('M'+str(i), INT)
    s['timer'] = Symbol('timer'+str(i), REAL)
    return s

# Função init
def init(s,V0):
    a=Equals(s['T'],Real(0))
    b=Equals(s['V'],Real(V0))
    c=Equals(s['v'],Real(V0))
    d=Equals(s['M'],START)
    e=Equals(s['timer'],Real(0))
    return And(a,b,c,d,e)

# Função transição
def trans(s, p):
    #UNTIMED
    start_free= And(
        Equals(s['M'],START),
        Equals(p['M'],FREE),
        Equals(p['V'],s['V']),
        Equals(p['v'],s['v']),
        Equals(p['T'],s['T']),
        Equals(p['timer'], Real(0))
    )

    free_stopping= And(
        Equals(s['M'],FREE),
        Equals(p['M'],STOPPING),
        Equals(p['V'],s['V']),
        Equals(p['v'],s['v']),
        Equals(p['T'],s['T']),
        Equals(p['timer'],Real(0)),
        Or(s['timer']>=tau, And(s['V'] <= erro, s['v'] <= erro))
    )
    
    stopping_stopped= And(
        Equals(s['M'],STOPPING),
        Equals(p['M'],STOPPED),
        Equals(p['T'],s['T']),
        Equals(p['timer'],Real(0)),
        s['v'] <= erro,
        s['V'] <= erro,
        s['v'] >= -erro,
        s['V'] >= -erro,
        Equals(p['V'], Real(0)),
        Equals(p['v'], Real(0))
    )
    
    stopping_blocked = And(
        Equals(s['M'],STOPPING),
        Equals(p['M'],BLOCKED),
        Equals(p['V'], s['V']),
        Equals(p['T'], s['T']),
        Equals(p['timer'],Real(0)),
        s['V'] - s['v'] <= erro,
        Equals(p['v'],p['V']),  # Correção: p['v'] deve ser igual a p['V']
        Or(
            p['V'] > erro
        )
    )
    
    blocked_free= And(
        Equals(s['M'],BLOCKED),
        Equals(p['M'],FREE),
        Equals(p['V'], s['V']),
        Equals(p['v'], s['v']),
        Equals(p['T'], s['T']),
        Equals(p['timer'], Real(0)),
        Or(s['timer']>=tau, s['V']<=erro, s['v']<=erro)
    )
    
    #TIMED
    free_free= And(
        Equals(s['M'],FREE),
        Equals(p['M'],FREE),
        Equals(p['V'] - s['V'], (-c*(s['V']-s['v'])-b)*dt),
        Equals(p['v'], s['v'] + (-a*P + c*(s['V']-s['v']))*dt),
        Equals(p['timer'], s['timer']+dt),
        Equals(p['T'], s['T']+dt),
        p['timer'] <= tau,
        Or(p['V']>erro, p['v']>erro)
    )
    
    stopping_stopping= And(
        Equals(s['M'],STOPPING),
        Equals(p['M'],STOPPING),
        Equals(p['timer'], Real(0)),
        Equals(p['T'], s['T']+dt),
        Equals(p['V']-s['V'], (-c_stopping*(s['V']-s['v'])-b)*dt),
        Equals(p['v']-s['v'], (-a*P + c_stopping * (s['V']-s['v']))*dt),
        s['V'] - s['v'] > erro,
        p['V'] <= s['V'],
        Or(s['V']>erro, s['v']>erro)
    )
    
    stopped_stopped=And(
        Equals(s['M'],STOPPED),
        Equals(p['M'],STOPPED),
        Equals(p['timer'], Real(0)),
        Equals(p['T'], s['T']+dt),
        Equals(p['V'], Real(0)),
        Equals(p['v'], Real(0))
        )
    
    blocked_blocked= And(
        Equals(s['M'],BLOCKED),
        Equals(p['M'],BLOCKED),
        Equals(p['T'],s['T']+dt),
        Equals(p['timer'],s['timer']+dt),
        s['V']-s['v'] <= erro,
        Equals(p['V'],s['V']+(-a_blocked*P-b)*dt),         
        Equals(p['v'],p['V']),
        p['timer'] <= tau,
        p['V']>0
    )
    
    return Or(start_free,free_free, free_stopping, stopping_stopping,
             stopping_stopped, stopping_blocked, blocked_blocked,
             blocked_free, stopped_stopped)

# Função print_vars
def print_vars(s,solver):
    print("MODE: ",end="")
    x = solver.get_py_value(s['M'])
    if x == 0:
        print("START")
    elif x == 1:
        print("FREE")
    elif x == 2:
        print("STOPPING")
    elif x == 3:
        print("BLOCKED")
    elif x == 4:
        print("STOPPED")
    print("V: ", end="")
    print(float(solver.get_py_value(s['V'])))
    print("v: ", end="")
    print(float(solver.get_py_value(s['v'])))
    print("T: ", end="")
    print(float(solver.get_py_value(s['T'])))
    print("timer: ", end="")
    print(float(solver.get_py_value(s['timer'])))
    print("")
    print("")    

# Função gera_traco
def gera_traco(declare,init,trans,k,V_in):
    with Solver(name="z3") as solver:
        traco=[declare(i) for i in range(k)]
        solver.add_assertion(init(traco[0],V_in))

        for i in range(k-1):
            solver.add_assertion(trans(traco[i],traco[i+1]))

            
        if solver.solve():
            i=0
            print("is sat")
            for s in traco:
                print("ESTADO: ",end="")
                print(i)
                print_vars(s,solver)
                i=i+1
        else:
            print("unsat")

# Propriedades
def timer(s):
    r=Implies(Or(Equals(s['M'],FREE),Equals(s['M'],BLOCKED)),s['timer']<=tau)
    return r

def stops(s):
    r=Equals(s['M'],STOPPED)
    return r

def always_decreasing(s,p):
    return Implies(s['T']<=p['T'],p['V']<=s['V'])

# Verificação de propriedades
def kinduction_always(declare,init,trans,inv,k,v_in):
    with Solver(name="z3") as solver:
        s = [declare(i) for i in range(k)]
        solver.add_assertion(init(s[0],v_in))
        for i in range(k-1):
            solver.add_assertion(trans(s[i],s[i+1]))
            
        for i in range(k):
            solver.push()
            solver.add_assertion(Not(inv(s[i])))
            if solver.solve():
                print(f"> Contradição! O invariante não se verifica nos k estados iniciais.")
                i=0
                for state in s:
                    print("ESTADO: ",end="")
                    print_vars(state,solver)
                    i=i+1
                return
            solver.pop()
        
        s2 = [declare(i+k) for i in range(k+1)]
        
        for i in range(k):
            solver.add_assertion(inv(s2[i]))
            solver.add_assertion(trans(s2[i],s2[i+1]))
        
        solver.add_assertion(init(s2[0],v_in))
        
        solver.add_assertion(Not(inv(s2[-1])))
        
        if solver.solve():
            print(f"> Contradição! O passo indutivo não se verifica.")
            return
        
        print(f"> A propriedade verifica-se por k-indução (k={k}).")

def bmc_always(declare,init,trans,inv,K,v_in):
    with Solver(name="z3") as solver:
        traco = [declare(i) for i in range(K)]
        solver.add_assertion(init(traco[0],v_in))
        
        for k in range(K):
            if k>0:
                solver.add_assertion(trans(traco[k-1], traco[k]))
            
            solver.push()
            solver.add_assertion(Not(inv(traco[k])))

            if solver.solve():
                print(f"> Invariante não se verifica nos primeiros {k+1} estados")                
            else:
                if k==K-1:
                    print(f"> Invariante verifica-se nos {K} estados")
                else:
                    solver.pop()

def bmc_eventually(declare,init,trans,prop,bound,v_in):
    with Solver(name="z3") as solver:
        states = [declare(i) for i in range(bound)]
        solver.add_assertion(init(states[0],v_in))
        
        for i in range(bound-1):
            solver.add_assertion(trans(states[i],states[i+1]))
            
        for i in range(bound):
            solver.push()
            solver.add_assertion(Not(prop(states[i])))
            
            if solver.solve():
                solver.pop()       
            else:
                print(f"> A propriedade occorre em {bound} estados")
                return
                
        print(f"> A propriedade nao ocorre em {bound} estados")
        return

def prove_decrasing_velocity(declare, init, trans, bound, v_in):
    with Solver(name="z3") as solver:
        states = [declare(i) for i in range(bound)]
        solver.add_assertion(init(states[0], v_in))

        for i in range(bound - 1):
            solver.add_assertion(trans(states[i], states[i + 1]))

        for i in range(bound - 1):
            solver.add_assertion(Not(always_decreasing(states[i],states[i+1])))
            if solver.solve():
                print(f"Contradição! A propriedade de diminuição constante de V não se verifica em {i+1} estados")
                return

        print(f"A propriedade de diminuição constante de V se verifica em {bound} estados")

# Testes
gera_traco(declare,init,trans,40,10)
kinduction_always(declare,init,trans,timer,30,10)
bmc_always(declare,init,trans,timer,30,10)
bmc_eventually(declare,init,trans,stops,50,10)
prove_decrasing_velocity(declare,init,trans,50,10)

unsat
> A propriedade verifica-se por k-indução (k=30).
> Invariante verifica-se nos 30 estados
